### 🔬 SCIANTIX Output viewer

This notebook loads the `output.txt` file from SCIANTIX simulations and provides an interactive interface to:
- Browse available variables
- Select X and Y axes from the output columns
- Visualize any pair of variables using interactive Plotly charts


In [ ]:
# !pip install pandas plotly ipywidgets
# !pip install --upgrade nbformat

import pandas as pd
import plotly.express as px
from ipywidgets import interact
import plotly.io as pio

# Ensure Plotly works in Jupyter notebooks
pio.renderers.default = 'notebook'

# Load the output file
df = pd.read_csv("output.txt", sep=r"\t+", engine="python")

# Show the first few rows
df.head()

In [ ]:
# List of columns from the file
columns = df.columns.tolist()

# Interactive plot using dropdowns
@interact(x=columns, y=columns)
def plot_sciantix(x='Time (h)', y='Fission gas release (/)'):
    """
    Plots an interactive line chart of the selected y-variable against x-variable
    from the SCIANTIX output dataframe.
    """
    fig = px.line(df, x=x, y=y, title=f"{y} vs {x}")
    fig.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact, widgets
import matplotlib.cm as cm

plt.style.use('seaborn-v0_8-dark')  # or 'ggplot', 'classic', 'bmh', etc.
print(plt.style.available)

# Get available variables (excluding time)
time_col = 'Time (h)'
data_cols = [col for col in df.columns if col != time_col]

@interact(
    variable=widgets.Dropdown(options=data_cols, description='Variable'),
    time_index=widgets.IntSlider(min=0, max=len(df)-1, step=1, description='Time Index', continuous_update=False)
)
def plot_grain(variable='Fission gas release (/)', time_index=0):
    """
    Plots a circular grain filled with a color representing the selected variable
    at a given time index.
    """
    value = df.iloc[time_index][variable]
    time = df.iloc[time_index][time_col]

    # Normalize color
    vmin, vmax = df[variable].min(), df[variable].max()
    if vmax == vmin:
        norm_val = 0.5
    else:
        norm_val = (value - vmin) / (vmax - vmin)
    color = cm.viridis(norm_val)

    # Plot
    fig, ax = plt.subplots(figsize=(4, 4))
    circle = plt.Circle((0, 0), radius=1, color=color, ec='black', linewidth=1.5)
    ax.add_patch(circle)
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_aspect('equal')
    ax.axis('off')
    
    ax.text(0, 0, f"{value:.2e}", ha='center', va='center', fontsize=12, color='white')

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    cbar = fig.colorbar(sm, ax=ax, shrink=0.8)

In [ ]:
@interact(
    variable=widgets.Dropdown(options=data_cols, description='Grain color'),
    time_index=widgets.IntSlider(min=0, max=len(df)-1, step=1, description='Time Index', continuous_update=False)
)
def plot_grain(variable='Fission gas release (/)', time_index=0):
    """
    Plots a circular grain with an intragranular bubble.
    - Outer grain color = selected variable
    - Bubble radius = actual bubble radius from SCIANTIX
    - Bubble color = optional (e.g., concentration)
    """
    row = df.iloc[time_index]
    time = row[time_col]
    grain_value = row[variable]

    # Outer grain color normalization
    vmin, vmax = df[variable].min(), df[variable].max()
    norm_val = 0.5 if vmax == vmin else (grain_value - vmin) / (vmax - vmin)
    grain_color = cm.viridis(norm_val)

    # 🔴 Intragranular bubble size
    bubble_radius_raw = row["Intragranular bubble radius (m)"]
    bubble_radius = bubble_radius_raw * 4e7  # scale for visibility

    # 🟠 Optional: bubble color from concentration
    bubble_conc = row["Intragranular bubble concentration (bub/m3)"]
    conc_min, conc_max = df["Intragranular bubble concentration (bub/m3)"].min(), df["Intragranular bubble concentration (bub/m3)"].max()
    bubble_norm = 0.5 if conc_max == conc_min else (bubble_conc - conc_min) / (conc_max - conc_min)
    bubble_color = cm.inferno(bubble_norm)

    # Plotting
    fig, ax = plt.subplots(figsize=(4, 4))

    # Grain
    grain = plt.Circle((0, 0), radius=1, color=grain_color, ec='black', linewidth=1.5)
    ax.add_patch(grain)

    # Bubble (off-center for visibility)
    bubble = plt.Circle((0.3, 0.3), radius=bubble_radius, color=bubble_color, ec='black', linewidth=1)
    ax.add_patch(bubble)

    # Layout
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_aspect('equal')
    ax.axis('off')

    # Grain colorbar
    sm1 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm1.set_array([])
    cbar1 = fig.colorbar(sm1, ax=ax, shrink=0.7, location='right')
    cbar1.set_label(variable)

    # Bubble colorbar
    sm2 = plt.cm.ScalarMappable(cmap='inferno', norm=plt.Normalize(vmin=conc_min, vmax=conc_max))
    sm2.set_array([])
    cbar2 = fig.colorbar(sm2, ax=ax, shrink=0.5, location='left')
    cbar2.set_label("Bubble concentration (bub/m3)")

    plt.title(f"{variable} at t = {time:.1f} h")
    plt.show()


In [ ]:
%matplotlib inline
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.cm as cm

@interact(
    variable=widgets.Dropdown(options=data_cols, description='Grain color'),
    time_index=widgets.IntSlider(min=0, max=len(df)-1, step=1, description='Time Index', continuous_update=False)
)
def plot_grain(variable='Fission gas release (/)', time_index=0):
    """
    Plots a circular grain with many intragranular bubbles.
    - Grain color = selected variable
    - Bubble size = actual bubble radius
    - Bubble count = scaled from concentration
    """
    row = df.iloc[time_index]
    time = row[time_col]
    grain_value = row[variable]

    # Grain color normalization
    vmin, vmax = df[variable].min(), df[variable].max()
    norm_val = 0.5 if vmax == vmin else (grain_value - vmin) / (vmax - vmin)
    grain_color = cm.viridis(norm_val)

    # Bubble data
    bubble_radius_raw = row["Intragranular bubble radius (m)"]
    bubble_conc = row["Intragranular bubble concentration (bub/m3)"]

    # 🧮 Estimate number of bubbles to show (cap at 50)
    bubble_count = min(50, int((bubble_conc / 1e+24) * 50))  # scale: 1e+24 → 50 bubbles

    # 📐 Scale bubble size for visibility
    scaled_radius = bubble_radius_raw * 1e8
    scaled_radius = max(scaled_radius, 0.01)

    # Bubble color based on concentration
    conc_min, conc_max = df["Intragranular bubble concentration (bub/m3)"].min(), df["Intragranular bubble concentration (bub/m3)"].max()
    bubble_norm = 0.5 if conc_max == conc_min else (bubble_conc - conc_min) / (conc_max - conc_min)
    bubble_color = cm.inferno(bubble_norm)

    # Plot setup
    fig, ax = plt.subplots(figsize=(6, 6))  # larger canvas
    plt.subplots_adjust(left=0.15, right=0.85, top=0.88, bottom=0.12)  # more breathing room

    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_aspect('equal')
    ax.axis('off')

    # Draw outer grain
    grain = plt.Circle((0, 0), radius=1, color=grain_color, ec='black', linewidth=1.5)
    ax.add_patch(grain)

    # Draw multiple bubbles randomly inside the grain
    rng = np.random.default_rng(seed=42)  # fixed seed for reproducibility
    for _ in range(bubble_count):
        while True:
            x, y = rng.uniform(-1, 1, size=2)
            if x**2 + y**2 < 0.95**2:  # stay inside grain
                break
        bubble = plt.Circle((x, y), radius=scaled_radius, color=bubble_color, ec='black', linewidth=0.5)
        ax.add_patch(bubble)

    # Colorbars
    sm1 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm1.set_array([])
    fig.colorbar(sm1, ax=ax, shrink=0.7, location='right').set_label(variable)

    sm2 = plt.cm.ScalarMappable(cmap='inferno', norm=plt.Normalize(vmin=conc_min, vmax=conc_max))
    sm2.set_array([])
    fig.colorbar(sm2, ax=ax, shrink=0.5, location='left').set_label("Bubble concentration (bub/m³)")

    plt.title(f"{variable} at t = {time:.1f} h", fontsize=14)
    plt.show()


In [ ]:
%matplotlib inline
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.cm as cm

# Setup figure
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)
ax.set_aspect('equal')
ax.axis('off')

# Draw grain
grain_circle = plt.Circle((0, 0), radius=1, ec='black', linewidth=1.5)
ax.add_patch(grain_circle)

# Prepare multiple bubble patches
bubble_patches = [plt.Circle((0, 0), radius=0.0) for _ in range(50)]
for patch in bubble_patches:
    ax.add_patch(patch)

title = ax.text(0.5, 1.05, "", transform=ax.transAxes, ha='center', fontsize=12)

# Precompute color ranges
grain_var = 'Fission gas release (/)'  # You can change this
bubble_conc_var = 'Intragranular bubble concentration (bub/m3)'
bubble_rad_var = 'Intragranular bubble radius (m)'

vmin, vmax = df[grain_var].min(), df[grain_var].max()
cmin, cmax = df[bubble_conc_var].min(), df[bubble_conc_var].max()

# Random bubble positions inside the grain
rng = np.random.default_rng(seed=42)
bubble_positions = []
while len(bubble_positions) < 50:
    x, y = rng.uniform(-1, 1, 2)
    if x**2 + y**2 <= 0.95**2:
        bubble_positions.append((x, y))

# Update function
def update(frame):
    row = df.iloc[frame]
    grain_val = row[grain_var]
    conc = row[bubble_conc_var]
    rad = row[bubble_rad_var]

    # Grain color
    norm = (grain_val - vmin) / (vmax - vmin) if vmax != vmin else 0.5
    grain_color = cm.viridis(norm)
    grain_circle.set_facecolor(grain_color)

    # Bubble appearance
    conc_norm = (conc - cmin) / (cmax - cmin) if cmax != cmin else 0.5
    bubble_color = cm.inferno(conc_norm)
    scaled_radius = max(rad * 1e8, 0.01)  # visual scale

    for i, patch in enumerate(bubble_patches):
        patch.set_center(bubble_positions[i])
        patch.set_radius(scaled_radius)
        patch.set_color(bubble_color)
        patch.set_edgecolor('black')
        patch.set_linewidth(0.5)

    title.set_text(f"{grain_var} at t = {row['Time (h)']:.1f} h")

# Create animation
anim = FuncAnimation(fig, update, frames=len(df), interval=150)

# Show in VS Code notebook
HTML(anim.to_jshtml())
